Si vas a ejecutar en la webapp:

In [15]:
!sudo apt update
!sudo apt install netcat-openbsd

Hit:1 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu noble InRelease
Hit:3 http://archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done3m
Building dependency tree... Done
Reading state information... Done
33 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
netcat-openbsd is already the newest version (1.226-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf


spark = SparkSession.builder.master('local[2]').getOrCreate()
#spark.conf.set('spark.cores.max', '1')
#spark.conf.set('spark.driver.memory','1000M')

sc = spark.sparkContext

spark

Primero, importamos StreamingContext, que es el principal punto de entrada para toda la funcionalidad de streaming. Creamos un StreamingContext local con dos hilos de ejecución, y un intervalo de lote de 1 segundo.

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = spark.sparkContext
ssc = StreamingContext(sc, 2)
sc.setCheckpointDir("checkpoint")

/usr/local/spark/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


-------------------------------------------
Time: 2024-10-16 16:27:10
-------------------------------------------

-------------------------------------------
Time: 2024-10-16 16:27:12
-------------------------------------------

-------------------------------------------
Time: 2024-10-16 16:27:14
-------------------------------------------

-------------------------------------------
Time: 2024-10-16 16:27:16
-------------------------------------------

-------------------------------------------
Time: 2024-10-16 16:27:18
-------------------------------------------

-------------------------------------------
Time: 2024-10-16 16:27:20
-------------------------------------------

-------------------------------------------
Time: 2024-10-16 16:27:22
-------------------------------------------
('hfjd', 1)
('ksdjf', 1)

-------------------------------------------
Time: 2024-10-16 16:27:24
-------------------------------------------
('hfjd', 1)
('ksdjf', 1)

------------------------------

Utilizando este contexto, podemos crear un DStream que represente los datos de streaming de una fuente TCP, especificados como nombre de host (por ejemplo, localhost) y puerto (por ejemplo, 1234).

Desde la máquina (`docker exec -it spark-lab-spark-1 /bin/bash`)
```bash
nc -l 127.0.0.1 1234
```

In [3]:
lines = ssc.socketTextStream("127.0.0.1", 1235)

El DStream línea representa el flujo de datos que se recibirá del servidor de datos. Cada registro en este DStream es una línea de texto. A continuación, queremos dividir las líneas por el espacio en palabras.

In [4]:
words = lines.flatMap(lambda line: line.split(" "))

flatMap es una operación de uno a muchos DStream que crea un nuevo DStream generando múltiples registros nuevos de cada registro en el DStream de origen. En este caso, cada línea se dividirá en múltiples palabras y el flujo de palabras se representa como las palabras DStream. A continuación, queremos contar estas palabras.

In [5]:
# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 10)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()
wordCounts.saveAsTextFiles("output/spark_streaming/streaming_")

Las palabras DStream son mapeadas más adelante (transformación uno a uno) a un DStream de (palabra, 1) pares, que luego se reduce para obtener la frecuencia de palabras en cada lote de datos. Finalmente, wordCounts.pprint() imprimirá algunos de los recuentos generados cada segundo.

Nótese que cuando estas líneas se ejecutan, Spark Streaming sólo establece el cómputo que realizará cuando se inicie, y aún no se ha iniciado un verdadero procesamiento. Para iniciar el procesamiento después de que todas las transformaciones han sido configuradas, finalmente llamamos

In [6]:
ssc.start()             # Start the computation

In [7]:
ssc.stop()